In [ ]:
##importing libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [ ]:
##initializing chatbot training
words=[]
classes = []
documents = []
ignore_words = ['?', '!','#','&','(',')','a','are']
data_file = open('dataset2.json').read()
intents = json.loads(data_file)

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents  
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
# sort in alphabetical order
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

# save them into pickle file
pickle.dump(words,open('dictionary.pkl','wb'))
pickle.dump(classes,open('pattern_classes.pkl','wb'))

In [ ]:
##Building Model
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. 
# X -> patterns, Y -> intents
train_x = list(training[:,0])
train_y = list(training[:,1])

### May not need to run the below cell, if chatbot_model.h5 already created

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
# softmax get highest probability -> only 1 output
model.add(Dense(len(train_y[0]), activation='softmax'))

# Model optimization
# Compile model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=35, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")
#print("predict ",model.predict(train_x, batch_size =5, verbose =1))
#print("predict class",model.predict_classes(train_x, batch_size =5, verbose =1))
#print("predict proba",model.predict_proba(train_x, batch_size =5, verbose =1))

In [ ]:
##Building Chatbot GUI
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('dataset2.json').read())
words = pickle.load(open('dictionary.pkl','rb'))
classes = pickle.load(open('pattern_classes.pkl','rb'))

In [ ]:
import re
QuestionFlag = "Havent ask"
specialcase = " "
category = "null" 
Storage_msg = " "
Remember_msg = " "
questionAns = []
interest = ["Accounting", "Broadcasting", "Food Science", "Psychology", "Public Relations", "Business Administration","Human Resource Management","Information Technology","Computer Science","Engineering","Architecture","Quantity Surveying","Hotel Management","Culinary Arts","Advertising","Journalism","Finance","English"]
questionArr = ["Q1","Q2","Q3","Q4","Q5","Q6","Q7","Q8","Q9","Q10","Q11","Q12","Q13","Q14","Q15","Q16","Q17","Q18"]
eduLevel = 0

# Convert word number to integer number
def word_to_number(word):
    from word2number import w2n
    return w2n.word_to_num(word)

def clean_up_sentence(sentence):
    global Remember_msg
    global category
    global specialcase
    global QuestionFlag
    global eduLevel
    global questionAns
    
    # Age - Education level e.g. Pre-U, Undergraduate, Postgraduate
    if (QuestionFlag == "Asked"):
        specialcase = "age"
        QuestionFlag = "null"
        if (sentence.isnumeric() == False):
            sentence = word_to_number(sentence)
        sentence = str(sentence)
        x = re.findall("[0-5][0-9]", sentence) 
        print("Age: ",x[0])
        if(int(x[0]) <= 17):
            category = "too young"
        elif(18 <= int(x[0]) <= 19 ):
            category = "pre-u"
            eduLevel = 1
        elif(20 <= int(x[0]) <= 23 ):
            category = "undergraduate"
            eduLevel = 2
        elif(int(x[0]) >= 24):
            category = "postgraduate"
            eduLevel = 3
            
    # Run Questions
    for x in range(len(interest)):
        if (QuestionFlag == questionArr[x]): 
            specialcase = questionArr[x]
            QuestionFlag = "null"
            if (sentence.isnumeric()):
                questionAns.append(int(sentence))
            print(questionAns)
    
    # All Questions Done, run GaussianNB, get the suitable course name 
    if (QuestionFlag == "finish question"):   
        QuestionFlag = "null"
        specialcase = "finish question" 
        
        import pandas as pd 
        df = pd.read_csv('nbDatasets.csv') 

        cols = list(df.columns)
        features = df[cols[1:-1]]
        result = df["course"]

        # Import LabelEncoder
        from sklearn import preprocessing
        #creating labelEncoder
        le = preprocessing.LabelEncoder()
        # Converting string labels into numbers.
        result_encoded=le.fit_transform(result)
        # print(result_encoded)


        #Import Gaussian Naive Bayes model
        from sklearn.naive_bayes import GaussianNB

        #Create a Gaussian Classifier
        model = GaussianNB()

        # Train the model using the training sets
        model.fit(features,result_encoded)
        
        # clean questionAns list in second run of questionnaire
        if len(questionAns) > 18:
            del questionAns [:18] 
            
        #Predict Output
        predicted= model.predict([questionAns])
        transform = le.inverse_transform(predicted)
        transform = transform[0]
        Remember_msg += " " + transform        
        sentence = sentence + " " + Remember_msg
        sentence_words = nltk.word_tokenize(sentence)
        sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        print(sentence_words)
        return sentence_words
              
    sentence = sentence + " " + Remember_msg
    print(sentence)
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print(sentence_words)
    return sentence_words

def words_bag(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for sentenceWord in sentence_words:
        for i,word in enumerate(words):
            if word == sentenceWord:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

def predict_class(sentence, model):
    p = words_bag(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    # get the highest one only -> Only one output
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents):
    global specialcase
    global Remember_msg
    print("-------ints: " ,ints)
    print("-------tags: ", ints[0]['intent'])
    tag = ints[0]['intent']
    list_of_intents = intents['intents']
    
    for i in list_of_intents:
        if(i['tag']== tag):
            if (eduLevel == 1 or eduLevel == 2 or eduLevel == 3):
                try:
                    result = i['responses'][eduLevel-1]
                except:
                    result = random.choice(i['responses'])
            else:
                result = random.choice(i['responses'])

            break
    return result

def chatbot_response(msg):
    global specialcase
    global QuestionFlag
    global questionAns
    global Storage_msg
    global Remember_msg
    global eduLevel
    ints = predict_class(msg, model)
    #print("edu leveL: ", eduLevel)
    res = getResponse(ints, intents)
    
    # Get Age
    if (res[-1] == '{'):
        if (QuestionFlag == "Havent ask"):
            QuestionFlag = "Asked"
            Storage_msg = msg
            res = "Oh before that, may i know your age please?"
        else:
            res = res[:-1]
    
    # Run the likert scale questions
    if (res[-1] == 'x'):
        Remember_msg = " " #clear the Remember_msg 
        QuestionFlag = "Q1"
        res = "Answer the question with scale 1 to 5, 1 - Most disagree, 2 - Disagree,  3 - Average,  4 - Agree,  5 - Most agree \nAre you a detail oriented, number sensitive, trustworthy, good in calculation person?"
        
    if (specialcase != "null"):
        if (specialcase == "age"):
            
            # Age and education level responses
            if (category == "too young"):
                res += " but you are too young for now, come consult me when your are > 17 years old"
            elif (category == "pre-u"):
                res += " and based on your age, i bet you are a Pre University student, hence I only display Pre University Courses for you"
                Remember_msg += " pre-u"
                eduLevel = 1
            elif (category == "undergraduate"):
                res += " and based on your age, i bet you are an Undergraduate student, hence I only display Undergraduate Courses for you "
                Remember_msg += " undergraduate"
                eduLevel = 2
            elif (category == "postgraduate"):
                res += " and based on your age, i bet you are a Postgraduate student, hence I only display Postgraduate Courses for you "
                Remember_msg += " postgraduate"
                eduLevel = 3
                
        # Ask Questions
        if (specialcase == "Q1"):
            QuestionFlag = "Q2"
            res = "Are you interested in broadcasting e.g. filming, cinematography, production and multimedia?"
        if (specialcase == "Q2"):
            QuestionFlag = "Q3"
            res = "Are you interested in chemistry, biology and composition of foods?"
        if (specialcase == "Q3"):
            QuestionFlag = "Q4"
            res = "Are you interested in helping people to solve their problems patiently?"
        if (specialcase == "Q4"):
            QuestionFlag = "Q5"
            res = "Are you interested in public relations which buidls mutually beneficial relationships between organization and their publics?"
        if (specialcase == "Q5"):
            QuestionFlag = "Q6"
            res = "Are you interested in operating a business?"
        if (specialcase == "Q6"):
            QuestionFlag = "Q7"
            res = "Are you interested in managing people where we called it as human resource management?"
        if (specialcase == "Q7"):
            QuestionFlag = "Q8"
            res = "Are you interested in coding, programming, computer security or develop software?"
        if (specialcase == "Q8"):
            QuestionFlag = "Q9"
            res = "Are you interested in analyze data, computer science or develop computer system application?"
        if (specialcase == "Q9"):
            QuestionFlag = "Q10"
            res = "Are you interested in engineering e.g. electronic, mechanical, mechatronic and telecommunication?"
        if (specialcase == "Q10"):
            QuestionFlag = "Q11"
            res = "Are you interested in interior design or architercture?"
        if (specialcase == "Q11"):
            QuestionFlag = "Q12"
            res = "Are you interested in managing the costs and budgets of a project?"
        if (specialcase == "Q12"):
            QuestionFlag = "Q13"
            res = "Are you interested in learning the technical skills in food and beverage, room management, managerial and personal skills?"
        if (specialcase == "Q13"):
            QuestionFlag = "Q14"
            res = "Are you interested in cooking?"
        if (specialcase == "Q14"):
            QuestionFlag = "Q15"
            res = "Are you interested in learning the skills of advertising, media and communication?"
        if (specialcase == "Q15"):
            QuestionFlag = "Q16"
            res = "Are you interested in become a journalists, writers or reporters?"
        if (specialcase == "Q16"):
            QuestionFlag = "Q17"
            res = "Are you interested in managing the financial assets?"
        if (specialcase == "Q17"):
            QuestionFlag = "Q18"
            res = "Are you interested in english education, english drama or english studies?"
        if (specialcase == "Q18"):
            QuestionFlag = "finish question"
            res = "Based on the questions you answered just now, here are the recommended course(s). Type ok to display.\n"
    
    print("remember msg: ",Remember_msg)
    print(specialcase)
    return res

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
import cv2
countRes = 0 
def send():
    global specialcase
    global Storage_msg
    global countRes
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        # User input
        ChatLog.insert(END, "You: " + msg + '\n\n')
        
        # Bot responses
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        
        # Display hot courses
        countRes += 1
        if countRes == 1:
            ChatLog.insert(END, "Bot: Here are some hot courses:  Accounting\n"+
            "\t\t\t       Interior design\n"+
            "\t\t\t       Data Science\n"+
            "\t\t\t       Broadcasting\n"+
            "\t\t\t       Business administration\n"+
            "\t\t\t       Food Science\n"+
            "\t\t\t       Finance\n"+
            "\t\t\t       Hotel Management\n"+
            "\t\t\t       Psychology" + '\n\n')
            ChatLog.insert(END, "Bot: May I know what course you interested in?" +'\n\n')

#         print(specialcase)
#        print(Storage_msg)
        if(specialcase == 'age'):
            specialcase = "null"
            res = chatbot_response(Storage_msg)
            ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

# Tkinter Initialization
base = Tk()
base.title("TARUC Course Recommender Bot")
base.geometry("545x500")
base.resizable(width=FALSE, height=FALSE)

# Background
img = cv2.imread("chat_bg.png")
cv2.imwrite("chat_bg.png",img)
background = PhotoImage(file="chat_bg.png")
background_Label = Label(base, image = background)
background_Label.place(x=0,y=0)
#background_Label.pack()

#Create Chat window
ChatLog = Text(base, bd=0, height="10", width="150", font="Arial")
ChatLog.config(foreground="#4361ee", font=("Verdana", 12 ))
ChatLog.insert(END, "Bot: Welcome to TARUC Course Recommender System." + '\n\n')
ChatLog.config(state=DISABLED)

#Create scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#4361ee", activebackground="#023e8a",fg='#ffffff',
                    command= send)

#Create the input box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=525,y=6, height=376)
ChatLog.place(x=6,y=6, height=386, width=520)
EntryBox.place(x=6, y=401, height=90, width=400)
SendButton.place(x=405, y=401, height=90)
base.mainloop()